In [5]:
# 마이리얼트립 크롤러 - 검색창 문제 수정 버전
# 작동하는 파일의 성공 요소들을 적용

import pandas as pd
import warnings, os, time, shutil, urllib, random, logging
from typing import Optional, Tuple, List, Dict, Any
warnings.filterwarnings(action='ignore')

from PIL import Image
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException, StaleElementReferenceException

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# 🔧 수정된 설정값 (작동하는 파일의 성공 요소 적용)
CONFIG = {
    "WAIT_TIMEOUT": 10,  # 원래 파일과 동일하게
    "RETRY_COUNT": 3,
    "MIN_DELAY": 3,      # 원래 파일과 동일하게
    "MAX_DELAY": 8,      # 원래 파일과 동일하게
    "POPUP_WAIT": 5,     # 원래 파일과 동일하게
    # 🚀 핵심 수정: 작동하는 파일의 User Agent 사용
    "USER_AGENT": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Whale/4.32.315.22 Safari/537.36",
    "SAVE_IMAGES": True,
    "SAVE_INTERMEDIATE": True,
    "MAX_PRODUCT_NAME_LENGTH": 30  # 원래 파일과 동일하게
}

# 검색 설정
CITIES_TO_SEARCH = ["방콕"]
PRODUCTS_PER_CITY = 3

# 도시별 정보 매핑
CITY_INFO = {
    "방콕": {"대륙": "아시아", "국가": "태국"},
    "도쿄": {"대륙": "아시아", "국가": "일본"},
    "오사카": {"대륙": "아시아", "국가": "일본"},
    "싱가포르": {"대륙": "아시아", "국가": "싱가포르"},
    "홍콩": {"대륙": "아시아", "국가": "홍콩"},
    "푸켓": {"대륙": "아시아", "국가": "태국"},
    "파타야": {"대륙": "아시아", "국가": "태국"},
    "치앙마이": {"대륙": "아시아", "국가": "태국"},
    "파리": {"대륙": "유럽", "국가": "프랑스"},
    "런던": {"대륙": "유럽", "국가": "영국"},
    "로마": {"대륙": "유럽", "국가": "이탈리아"},
    "뉴욕": {"대륙": "북미", "국가": "미국"},
    "로스앤젤레스": {"대륙": "북미", "국가": "미국"},
}

# === 유틸리티 함수들 ===
def print_progress(current: int, total: int, city_name: str, status: str = "진행중"):
    """진행률 표시"""
    percentage = (current / total) * 100
    bar_length = 30
    filled_length = int(bar_length * current // total)
    bar = '█' * filled_length + '░' * (bar_length - filled_length)
    
    emoji = "🔍" if status == "진행중" else "✅" if status == "완료" else "❌"
    
    print(f"\n{emoji} 진행률: [{bar}] {percentage:.1f}% ({current}/{total})")
    print(f"📍 현재 작업: {city_name} - {status}")
    print("-" * 50)

def get_city_info(city_name: str) -> Tuple[str, str]:
    """도시 정보 조회"""
    info = CITY_INFO.get(city_name, {"대륙": "기타", "국가": "기타"})
    return info["대륙"], info["국가"]

def make_safe_filename(filename: str) -> str:
    """안전한 파일명 생성"""
    if not filename:
        return "기본파일명"
    
    safe_filename = str(filename)
    unsafe_chars = ['/', '\\', ':', '*', '?', '"', '<', '>', '|', '\n', '\r', '\t']
    for char in unsafe_chars:
        safe_filename = safe_filename.replace(char, '_')
    
    if len(safe_filename) > 200:
        safe_filename = safe_filename[:200]
    
    return safe_filename.strip() or "기본파일명"

def retry_operation(func, operation_name: str, max_retries: int = None):
    """재시도 메커니즘"""
    if max_retries is None:
        max_retries = CONFIG["RETRY_COUNT"]
    
    for attempt in range(max_retries):
        try:
            return func()
        except (TimeoutException, NoSuchElementException, WebDriverException) as e:
            if attempt == max_retries - 1:
                logger.error(f"{operation_name} 최종 실패: {type(e).__name__}")
                raise e
            
            wait_time = min(2 ** attempt, 5)
            logger.warning(f"{operation_name} 재시도 {attempt + 1}/{max_retries}, {wait_time}초 후")
            time.sleep(wait_time)
        except Exception as e:
            logger.error(f"{operation_name} 예상치 못한 오류: {e}")
            raise e

# === 🔧 작동하는 파일의 드라이버 설정 방식 적용 ===
def make_user_agent(ua, is_mobile):
    """작동하는 파일의 User Agent 생성 함수"""
    from user_agents import parse
    
    user_agent = parse(ua)
    model = user_agent.device.model
    platform = user_agent.os.family
    platform_version = user_agent.os.version_string + ".0.0"
    version = user_agent.browser.version[0]
    ua_full_version = user_agent.browser.version_string
    architecture = "x86"
    
    print(platform)
    if is_mobile:
        platform_info = "Linux armv8l"
        architecture = ""
    else:  # Windows
        platform_info = "Win32"
        model = ""
    
    RET_USER_AGENT = {
        "appVersion": ua.replace("Mozilla/", ""),
        "userAgent": ua,
        "platform": f"{platform_info}",
        "acceptLanguage": "ko-KR, kr, en-US, en",
        "userAgentMetadata": {
            "brands": [
                {"brand": "Google Chrome", "version": f"{version}"},
                {"brand": "Chromium", "version": f"{version}"},
                {"brand": " Not A;Brand", "version": "99"}
            ],
            "fullVersionList": [
                {"brand": "Google Chrome", "version": f"{version}"},
                {"brand": "Chromium", "version": f"{version}"},
                {"brand": " Not A;Brand", "version": "99"}
            ],
            "fullVersion": f"{ua_full_version}",
            "platform": platform,
            "platformVersion": platform_version,
            "architecture": architecture,
            "model": model,
            "mobile": is_mobile
        }
    }
    return RET_USER_AGENT

def generate_random_geolocation():
    """지리적 위치 생성"""
    ltop_lat = 37.75415601640249
    ltop_long = 126.86767642302573
    rbottom_lat = 37.593829172663945
    rbottom_long = 127.15276051439332

    targetLat = random.uniform(rbottom_lat, ltop_lat)
    targetLong = random.uniform(ltop_long, rbottom_long)
    return {"latitude": targetLat, "longitude": targetLong, "accuracy": 100}

def setup_driver():
    """🔧 작동하는 파일의 드라이버 설정 방식 적용"""
    import chromedriver_autoinstaller
    import undetected_chromedriver as uc
    
    try:
        chromedriver_autoinstaller.install()
        
        options = uc.ChromeOptions()
        
        # 🚀 핵심: 작동하는 파일과 동일한 User Agent 설정
        UA = CONFIG["USER_AGENT"]
        options.add_argument(f"--user-agent={UA}")
        
        # 쿠키 폴더 설정 (작동하는 파일과 동일한 방식)
        rand_user_folder = random.randrange(1, 100)
        raw_path = os.path.abspath("cookies")
        try:
            shutil.rmtree(raw_path)
        except:
            pass
        os.makedirs(raw_path, exist_ok=True)
        user_cookie_name = f"{raw_path}/{rand_user_folder}"
        if os.path.exists(user_cookie_name) == False:
            os.makedirs(user_cookie_name, exist_ok=True)
        
        # 드라이버 생성
        driver = uc.Chrome(user_data_dir=user_cookie_name, options=options)
        logger.info("크롬 드라이버 실행 성공!")
        
        # 🚀 핵심: 작동하는 파일과 동일한 CDP 명령어 적용
        UA_Data = make_user_agent(UA, False)
        driver.execute_cdp_cmd("Network.setUserAgentOverride", UA_Data)
        
        GEO_DATA = generate_random_geolocation()
        driver.execute_cdp_cmd("Emulation.setGeolocationOverride", GEO_DATA)
        driver.execute_cdp_cmd("Emulation.setUserAgentOverride", UA_Data)
        driver.execute_cdp_cmd("Emulation.setNavigatorOverrides", {"platform": "Linux armv8l"})
        
        logger.info("CDP 명령어 설정 완료")
        return driver
        
    except Exception as e:
        logger.error(f"드라이버 설정 실패: {e}")
        raise RuntimeError(f"드라이버 실행 실패: {e}")

# === 🔧 작동하는 파일의 검색 로직 적용 ===
def search_city_fixed(driver, city_name: str) -> bool:
    """작동하는 파일의 검색 방식 적용"""
    logger.info(f"'{city_name}' 검색 시작")
    
    # 1. 메인 페이지로 이동
    def go_to_main_page():
        driver.get("https://www.myrealtrip.com/experiences/")
        time.sleep(random.uniform(CONFIG["MIN_DELAY"], CONFIG["MAX_DELAY"]))
        return True
    
    retry_operation(go_to_main_page, "메인 페이지 이동")
    logger.info("마이리얼트립 페이지 열기 완료")
    
    # 2. 🔧 작동하는 파일의 검색창 셀렉터 사용
    def find_and_fill_search():
        logger.info(f"'{city_name}' 검색창 찾는 중...")
        search_selectors = [
            (By.CSS_SELECTOR, "input[placeholder*='어디로']"),
            (By.CSS_SELECTOR, "input[type='text']"),
            (By.XPATH, "//input[contains(@placeholder, '어디로')]"),
            (By.XPATH, "/html/body/main/div/div[2]/section[1]/div[1]/div/div/input")  # 작동하는 파일의 절대 XPath
        ]

        search_input = None
        for selector_type, selector_value in search_selectors:
            try:
                search_input = WebDriverWait(driver, CONFIG["WAIT_TIMEOUT"]).until(
                    EC.presence_of_element_located((selector_type, selector_value))
                )
                logger.info("✅ 검색창을 찾았습니다!")
                break
            except TimeoutException:
                continue

        if not search_input:
            raise NoSuchElementException("검색창을 찾을 수 없습니다")

        # 검색어 입력
        search_input.clear()
        search_input.send_keys(city_name)
        time.sleep(random.uniform(CONFIG["MIN_DELAY"], CONFIG["MIN_DELAY"]+2))
        logger.info(f"'{city_name}' 키워드 입력 완료")
        return True
    
    retry_operation(find_and_fill_search, "검색창 입력")

    # 3. 🔧 작동하는 파일의 검색 버튼 클릭 방식 사용 (Enter 키 대신)
    def click_search_button():
        logger.info("🔎 검색 버튼 찾는 중...")
        search_button_selectors = [
            (By.CSS_SELECTOR, "button[type='submit']"),
            (By.CSS_SELECTOR, ".search-btn"),
            (By.XPATH, "//button[contains(@class, 'search')]"),
            (By.XPATH, "//img[contains(@alt, '검색')]//parent::*"),
            (By.XPATH, "/html/body/main/div/div[2]/section[1]/div[1]/div/div/div/img")  # 작동하는 파일의 절대 XPath
        ]

        search_clicked = False
        for selector_type, selector_value in search_button_selectors:
            try:
                search_button = WebDriverWait(driver, CONFIG["WAIT_TIMEOUT"]).until(
                    EC.element_to_be_clickable((selector_type, selector_value))
                )
                search_button.click()
                logger.info("✅ 검색 버튼 클릭 성공!")
                search_clicked = True
                time.sleep(random.uniform(CONFIG["MIN_DELAY"], CONFIG["MAX_DELAY"]))
                break
            except TimeoutException:
                continue

        if not search_clicked:
            raise NoSuchElementException("검색 버튼을 찾을 수 없습니다")
        return True
    
    retry_operation(click_search_button, "검색 버튼 클릭")
    
    return True

def handle_popups_fixed(driver) -> bool:
    """작동하는 파일의 팝업 처리 방식"""
    popup_selectors = [
        (By.CSS_SELECTOR, ".popup-close"),
        (By.CSS_SELECTOR, ".modal-close"),
        (By.XPATH, "//button[contains(@aria-label, '닫기')]"),
        (By.XPATH, "//button[contains(text(), '닫기')]"),
        (By.XPATH, "/html/body/div[15]/div[2]/button")  # 작동하는 파일의 절대 XPath
    ]

    popup_closed = False
    for selector_type, selector_value in popup_selectors:
        try:
            popup_button = WebDriverWait(driver, CONFIG["POPUP_WAIT"]).until(
                EC.element_to_be_clickable((selector_type, selector_value))
            )
            popup_button.click()
            logger.info("✅ 팝업창을 닫았습니다.")
            popup_closed = True
            time.sleep(random.uniform(1, 4))
            break
        except TimeoutException:
            continue

    if not popup_closed:
        logger.info("ℹ️ 팝업창이 없거나 이미 닫혀있습니다.")
    return True

def click_view_all_fixed(driver) -> bool:
    """작동하는 파일의 전체 상품 보기 방식"""
    logger.info("📋 전체 상품 보기 버튼 찾는 중...")
    view_all_selectors = [
        (By.XPATH, "//button[contains(text(), '전체')]"),
        (By.XPATH, "//span[contains(text(), '전체')]//parent::button"),
        (By.CSS_SELECTOR, "button[aria-label*='전체']"),
        (By.XPATH, "/html/body/div[4]/div[2]/div/div/div/span[21]/button")  # 작동하는 파일의 절대 XPath
    ]

    view_all_clicked = False
    for selector_type, selector_value in view_all_selectors:
        try:
            view_all_button = WebDriverWait(driver, CONFIG["WAIT_TIMEOUT"]).until(
                EC.element_to_be_clickable((selector_type, selector_value))
            )
            view_all_button.click()
            logger.info("✅ 전체 상품 보기 클릭 성공!")
            view_all_clicked = True
            time.sleep(random.uniform(CONFIG["MIN_DELAY"], CONFIG["MIN_DELAY"]+3))
            break
        except TimeoutException:
            continue

    if not view_all_clicked:
        logger.info("⚠️ 전체 상품 보기 버튼을 찾을 수 없습니다. 현재 상품으로 진행...")
    return True

def crawl_city_fixed(driver, city_name: str, city_index: int, total_cities: int) -> List[Dict]:
    """수정된 도시별 크롤링"""
    print_progress(city_index, total_cities, city_name, "진행중")
    
    continent, country = get_city_info(city_name)
    logger.info(f"{city_name} 크롤링 시작 - 대륙: {continent}, 국가: {country}")
    
    city_results = []
    
    try:
        # 1. 도시 검색 수행 (수정된 방식)
        retry_operation(lambda: search_city_fixed(driver, city_name), "도시 검색")
        
        # 2. 팝업 처리
        try:
            handle_popups_fixed(driver)
        except Exception as e:
            logger.info(f"팝업 처리 중 오류 (무시됨): {type(e).__name__}")
        
        # 3. 전체 상품 보기
        try:
            click_view_all_fixed(driver)
        except Exception as e:
            logger.info(f"전체 상품 보기 처리 중 오류 (무시됨): {type(e).__name__}")
        
        # 4. 첫 번째 상품 클릭 (작동하는 파일의 셀렉터 사용)
        def click_first_product():
            logger.info("🎯 첫 번째 상품 찾는 중...")
            first_product_selectors = [
                (By.CSS_SELECTOR, "a[href*='/experiences/'] img"),
                (By.CSS_SELECTOR, ".product-item:first-child img"),
                (By.XPATH, "//a[contains(@href, '/experiences/')][1]//img"),
                (By.XPATH, "/html/body/div[4]/div[2]/div/div/div[2]/main/div/div[3]/a[1]/div/div/img")  # 작동하는 파일의 절대 XPath
            ]

            product_clicked = False
            for selector_type, selector_value in first_product_selectors:
                try:
                    first_product = WebDriverWait(driver, CONFIG["WAIT_TIMEOUT"]).until(
                        EC.element_to_be_clickable((selector_type, selector_value))
                    )
                    first_product.click()
                    logger.info("✅ 첫 번째 상품 클릭 성공!")
                    product_clicked = True
                    time.sleep(random.uniform(CONFIG["MIN_DELAY"], CONFIG["MAX_DELAY"]))
                    break
                except TimeoutException:
                    continue

            if not product_clicked:
                raise NoSuchElementException("상품을 찾을 수 없습니다")
            return True
        
        retry_operation(click_first_product, "첫 번째 상품 클릭")
        
        # 5. 상품 정보 수집 (기본적인 정보만)
        product_data = {
            '번호': city_index + 1,
            '대륙': continent,
            '국가': country,
            '도시': city_name,
            '상품명': '정보 없음',
            '가격': '정보 없음',
            '평점': '정보 없음',
            '리뷰수': '',
            '언어': '',
            '이미지_경로': '처리 안됨',
            'URL': driver.current_url,
            '수집_시간': time.strftime('%Y-%m-%d %H:%M:%S'),
            '상태': '기본수집'
        }
        
        # 상품명 수집 시도
        try:
            title_selectors = [
                (By.CSS_SELECTOR, "h1"),
                (By.CSS_SELECTOR, ".product-title"),
                (By.XPATH, "//h1[contains(@class, 'title')]"),
                (By.XPATH, "/html/body/div[1]/main/div[1]/section/div[1]/h1")  # 작동하는 파일의 절대 XPath
            ]
            
            for selector_type, selector_value in title_selectors:
                try:
                    title_element = WebDriverWait(driver, CONFIG["WAIT_TIMEOUT"]).until(
                        EC.presence_of_element_located((selector_type, selector_value))
                    )
                    product_data['상품명'] = title_element.text
                    product_data['상태'] = '완전수집'
                    break
                except TimeoutException:
                    continue
        except Exception as e:
            logger.warning(f"상품명 수집 실패: {e}")
        
        city_results.append(product_data)
        
        print_progress(city_index + 1, total_cities, f"{city_name} (1개 상품)", "완료")
        logger.info(f"{city_name} 크롤링 완료")
        
    except Exception as e:
        logger.error(f"{city_name} 크롤링 중 오류: {e}")
        print_progress(city_index + 1, total_cities, city_name, "실패")
        
        # 실패시에도 기본 정보 저장
        error_result = {
            '번호': city_index + 1,
            '대륙': continent,
            '국가': country,
            '도시': city_name,
            '상품명': '크롤링 실패',
            '가격': '정보 없음',
            '평점': '정보 없음',
            '리뷰수': '',
            '언어': '',
            '이미지_경로': '처리 실패',
            'URL': '정보 없음',
            '수집_시간': time.strftime('%Y-%m-%d %H:%M:%S'),
            '상태': f'크롤링실패({type(e).__name__})'
        }
        city_results.append(error_result)
    
    return city_results

def main():
    """메인 실행 함수"""
    logger.info("마이리얼트립 크롤러 시작 - 검색창 문제 수정 버전")
    print("🔧 검색창 문제 수정 버전 - 작동하는 파일의 성공 요소 적용")
    
    all_results = []
    driver = None
    start_time = time.time()
    
    try:
        # 이미지 폴더 초기화
        if CONFIG["SAVE_IMAGES"]:
            img_folder = os.path.join(os.getcwd(), "myrealtripthumb_img")
            try:
                if os.path.exists(img_folder):
                    shutil.rmtree(img_folder)
                logger.info("기존 이미지 폴더 삭제 완료")
            except Exception as e:
                logger.warning(f"기존 이미지 폴더 삭제 실패: {e}")
            
            os.makedirs(img_folder, exist_ok=True)
            logger.info("이미지 폴더 생성 완료")
        
        # 🔧 수정된 드라이버 설정 (작동하는 파일 방식)
        print("🚀 Selenium 버전 확인 중...")
        import selenium
        print(f"🔧 Selenium 버전: {selenium.__version__}")
        
        driver = setup_driver()
        
        # 크롤링 시작 정보
        print("=" * 60)
        print(f"🌏 총 {len(CITIES_TO_SEARCH)}개 도시 검색 시작!")
        print(f"검색할 도시들: {', '.join(CITIES_TO_SEARCH)}")
        print(f"📊 현재 저장된 결과 수: {len(all_results)}개")
        print(f"⚙️ 설정: 재시도 {CONFIG['RETRY_COUNT']}회, 타임아웃 {CONFIG['WAIT_TIMEOUT']}초")
        print(f"🖼️ 이미지 저장: {'활성화' if CONFIG['SAVE_IMAGES'] else '비활성화'}")
        print(f"🔧 User Agent: {CONFIG['USER_AGENT'][:50]}...")
        print("=" * 60)
        
        # 각 도시별로 크롤링 실행
        successful_cities = 0
        
        for city_index, city_name in enumerate(CITIES_TO_SEARCH):
            try:
                city_results = crawl_city_fixed(driver, city_name, city_index, len(CITIES_TO_SEARCH))
                
                if city_results:
                    all_results.extend(city_results)
                    successful_cities += 1
                
                # 다음 검색을 위한 휴식 (마지막 도시 제외)
                if city_index < len(CITIES_TO_SEARCH) - 1:
                    wait_time = random.uniform(5, 10)
                    logger.info(f"다음 검색까지 {wait_time:.1f}초 대기")
                    time.sleep(wait_time)
                    
            except Exception as e:
                logger.error(f"{city_name} 크롤링 중 예상치 못한 오류: {e}")
                continue
        
        # 실행 시간 계산
        execution_time = time.time() - start_time
        
        # 최종 결과 처리
        print("\n" + "=" * 60)
        print("🎉 모든 도시 검색 완료!")
        print("=" * 60)
        
        if all_results:
            print(f"\n📊 총 {len(all_results)}개 상품 정보를 수집했습니다:")
            print("-" * 80)
            
            # 결과 요약 출력
            for result in all_results:
                print(f"{result['번호']}. {result['대륙']} > {result['국가']} > {result['도시']}")
                print(f"   상품명: {result['상품명']}")
                print(f"   가격: {result['가격']}")
                print(f"   평점: {result['평점']}")
                print(f"   상태: {result['상태']}")
                print(f"   URL: {result['URL']}")
                print("-" * 80)
            
            # CSV 파일로 저장
            try:
                df = pd.DataFrame(all_results)
                timestamp = time.strftime('%Y%m%d_%H%M%S')
                filename = f"myrealtrip_검색창수정_최종결과_{timestamp}.csv"
                df.to_csv(filename, index=False, encoding='utf-8-sig')
                
                print(f"\n💾 최종 결과가 '{filename}' 파일로 저장되었습니다!")
                print(f"📁 파일 위치: {os.path.abspath(filename)}")
                print("💡 엑셀에서 바로 열어볼 수 있습니다!")
                
                # CSV 파일 내용 미리보기
                print(f"\n📋 CSV 파일 내용 미리보기:")
                print(df.to_string(index=False, max_colwidth=30))
                
                # 상세 통계 정보
                print(f"\n📈 수집 통계:")
                print(f"   🌏 대륙별 분포: {df['대륙'].value_counts().to_dict()}")
                print(f"   🏙️ 국가별 분포: {df['국가'].value_counts().to_dict()}")
                print(f"   📊 수집 상태별 분포: {df['상태'].value_counts().to_dict()}")
                
                # 데이터 품질 분석
                total_products = len(df)
                quality_stats = {
                    '상품명': len(df[df['상품명'] != '정보 없음']),
                    '가격': len(df[df['가격'] != '정보 없음']),
                    '평점': len(df[df['평점'] != '정보 없음']),
                    '리뷰수': len(df[df['리뷰수'] != '']),
                    '언어': len(df[df['언어'] != '']),
                }
                
                print(f"   📋 데이터 품질 분석:")
                for field, count in quality_stats.items():
                    percentage = count / total_products * 100
                    print(f"      {field}: {count}/{total_products} ({percentage:.1f}%)")
                
                # 전체 성공률
                successful_products = len(df[df['상태'].isin(['완전수집', '기본수집'])])
                overall_success_rate = successful_products / total_products * 100
                print(f"   ✅ 전체 수집 성공률: {successful_products}/{total_products} ({overall_success_rate:.1f}%)")
                
                # 수정 사항 요약
                print(f"\n🔧 적용된 수정 사항:")
                print(f"   ✅ User Agent: Chrome 136 + Whale (작동하는 파일과 동일)")
                print(f"   ✅ 드라이버 설정: CDP 명령어 적용")
                print(f"   ✅ 검색 방식: 검색 버튼 클릭 (Enter 키 대신)")
                print(f"   ✅ 셀렉터: 절대 XPath 포함으로 강화")
                print(f"   ✅ 타임아웃: 10초 (작동하는 파일과 동일)")
                
                logger.info(f"CSV 파일 저장 완료: {filename}")
                
            except Exception as e:
                logger.error(f"CSV 파일 저장 중 오류: {e}")
                print(f"\n❌ CSV 파일 저장 중 오류: {e}")
        
        else:
            print("\n❌ 수집된 결과가 없습니다.")
            print("💡 수정 사항이 적용되었는지 확인해보세요:")
            print("   1. User Agent가 작동하는 파일과 동일한지")
            print("   2. CDP 명령어가 제대로 실행되었는지")
            print("   3. 검색 버튼 클릭 방식이 적용되었는지")
            print("💡 네트워크 연결이나 웹사이트 접근성도 확인해보세요.")
            logger.warning("수집된 데이터가 없음")
        
        # 최종 요약
        print(f"\n🏁 크롤링 완료!")
        print(f"⏰ 총 실행 시간: {execution_time/60:.1f}분")
        print(f"✅ 성공한 도시: {successful_cities}개")
        print(f"❌ 실패한 도시: {len(CITIES_TO_SEARCH) - successful_cities}개")
        print(f"📊 도시별 성공률: {successful_cities/len(CITIES_TO_SEARCH)*100:.1f}%")
        print(f"🔧 검색창 문제: 수정됨 ✅")
        print("=" * 60)
        
        logger.info(f"크롤링 완료 - 성공률: {successful_cities}/{len(CITIES_TO_SEARCH)}")
        
    except KeyboardInterrupt:
        logger.info("사용자에 의해 중단됨")
        print("\n⚠️ 사용자에 의해 중단되었습니다.")
        print("💾 지금까지 수집된 데이터를 저장합니다...")
        
        # 중단 시에도 결과 저장
        if all_results:
            try:
                df = pd.DataFrame(all_results)
                timestamp = time.strftime('%Y%m%d_%H%M%S')
                filename = f"myrealtrip_중단저장_{timestamp}.csv"
                df.to_csv(filename, index=False, encoding='utf-8-sig')
                print(f"📁 중단 결과 저장 완료: {filename}")
            except Exception as e:
                logger.error(f"중단 저장 실패: {e}")
        
    except Exception as e:
        logger.error(f"프로그램 실행 중 치명적 오류: {e}")
        print(f"\n❌ 프로그램 실행 중 오류: {type(e).__name__}: {e}")
        print("🔧 다음 사항들을 확인해보세요:")
        print("   1. user_agents 라이브러리가 설치되어 있는지")
        print("   2. 인터넷 연결이 안정적인지")
        print("   3. Chrome 브라우저가 최신 버전인지")
        
        # 오류 발생시에도 수집된 데이터가 있다면 저장
        if all_results:
            try:
                df = pd.DataFrame(all_results)
                timestamp = time.strftime('%Y%m%d_%H%M%S')
                filename = f"myrealtrip_오류저장_{timestamp}.csv"
                df.to_csv(filename, index=False, encoding='utf-8-sig')
                print(f"📁 오류 발생 전까지의 데이터 저장: {filename}")
            except Exception:
                pass
        
    finally:
        # 드라이버 안전 종료
        if driver:
            try:
                driver.quit()
                logger.info("웹드라이버 정상 종료")
                print("🚪 웹드라이버 정상 종료")
            except Exception as e:
                logger.warning(f"웹드라이버 종료 중 오류: {e}")
                print("⚠️ 웹드라이버 종료 중 오류 (무시됨)")
        
        # 임시 파일 정리
        try:
            cookies_path = os.path.join(os.getcwd(), "cookies")
            if os.path.exists(cookies_path):
                shutil.rmtree(cookies_path)
                logger.info("임시 쿠키 폴더 정리 완료")
        except Exception as e:
            logger.warning(f"임시 파일 정리 실패: {e}")
        
        print("🎯 모든 작업이 완료되었습니다!")
        print("🔧 검색창 문제가 해결되었는지 확인해주세요!")
        logger.info("프로그램 종료")

# 결과 저장소 초기화
all_results = []
print("🔄 결과 저장소 초기화 완료")

# 프로그램 실행
if __name__ == "__main__":
    main()

2025-07-13 01:34:38,802 - INFO - 마이리얼트립 크롤러 시작 - 검색창 문제 수정 버전
2025-07-13 01:34:38,806 - INFO - 기존 이미지 폴더 삭제 완료
2025-07-13 01:34:38,809 - INFO - 이미지 폴더 생성 완료
2025-07-13 01:34:38,977 - INFO - Chromedriver is already installed.


🔄 결과 저장소 초기화 완료
🔧 검색창 문제 수정 버전 - 작동하는 파일의 성공 요소 적용
🚀 Selenium 버전 확인 중...
🔧 Selenium 버전: 4.25.0


2025-07-13 01:34:45,991 - INFO - 크롬 드라이버 실행 성공!
2025-07-13 01:34:46,074 - INFO - CDP 명령어 설정 완료
2025-07-13 01:34:46,076 - INFO - 방콕 크롤링 시작 - 대륙: 아시아, 국가: 태국
2025-07-13 01:34:46,078 - INFO - '방콕' 검색 시작


Windows
🌏 총 1개 도시 검색 시작!
검색할 도시들: 방콕
📊 현재 저장된 결과 수: 0개
⚙️ 설정: 재시도 3회, 타임아웃 10초
🖼️ 이미지 저장: 활성화
🔧 User Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWeb...

🔍 진행률: [░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░] 0.0% (0/1)
📍 현재 작업: 방콕 - 진행중
--------------------------------------------------


2025-07-13 01:35:03,959 - INFO - 마이리얼트립 페이지 열기 완료
2025-07-13 01:35:03,962 - INFO - '방콕' 검색창 찾는 중...
2025-07-13 01:35:34,923 - INFO - ✅ 검색창을 찾았습니다!
2025-07-13 01:35:39,099 - INFO - '방콕' 키워드 입력 완료
2025-07-13 01:35:39,100 - INFO - 🔎 검색 버튼 찾는 중...
2025-07-13 01:36:27,331 - INFO - ✅ 검색 버튼 클릭 성공!
2025-07-13 01:36:52,604 - INFO - ✅ 팝업창을 닫았습니다.
2025-07-13 01:36:54,900 - INFO - 📋 전체 상품 보기 버튼 찾는 중...
2025-07-13 01:37:19,768 - INFO - ✅ 전체 상품 보기 클릭 성공!
2025-07-13 01:37:24,721 - INFO - 🎯 첫 번째 상품 찾는 중...
2025-07-13 01:38:01,344 - INFO - ✅ 첫 번째 상품 클릭 성공!
2025-07-13 01:38:09,103 - INFO - 방콕 크롤링 완료
2025-07-13 01:38:09,198 - INFO - CSV 파일 저장 완료: myrealtrip_검색창수정_최종결과_20250713_013809.csv
2025-07-13 01:38:09,198 - INFO - 크롤링 완료 - 성공률: 1/1
2025-07-13 01:38:09,200 - INFO - 웹드라이버 정상 종료
2025-07-13 01:38:09,257 - WARNING - 임시 파일 정리 실패: [WinError 32] 다른 프로세스가 파일을 사용 중이기 때문에 프로세스가 액세스 할 수 없습니다: 'c:\\Users\\redsk\\OneDrive\\デスクトップ\\mikael_project\\cookies\\68\\Default\\Account Web Data'
2025-07-13 01:38:09,259 - 


✅ 진행률: [██████████████████████████████] 100.0% (1/1)
📍 현재 작업: 방콕 (1개 상품) - 완료
--------------------------------------------------

🎉 모든 도시 검색 완료!

📊 총 1개 상품 정보를 수집했습니다:
--------------------------------------------------------------------------------
1. 아시아 > 태국 > 방콕
   상품명: 벨럭, 방콕 호텔→수완나품 공항/ 방콕 호텔간 짐 배송 서비스
   가격: 정보 없음
   평점: 정보 없음
   상태: 완전수집
   URL: https://experiences.myrealtrip.com/products/3443042
--------------------------------------------------------------------------------

💾 최종 결과가 'myrealtrip_검색창수정_최종결과_20250713_013809.csv' 파일로 저장되었습니다!
📁 파일 위치: c:\Users\redsk\OneDrive\デスクトップ\mikael_project\myrealtrip_검색창수정_최종결과_20250713_013809.csv
💡 엑셀에서 바로 열어볼 수 있습니다!

📋 CSV 파일 내용 미리보기:
 번호  대륙 국가 도시                            상품명    가격    평점 리뷰수 언어 이미지_경로                            URL               수집_시간   상태
  1 아시아 태국 방콕 벨럭, 방콕 호텔→수완나품 공항/ 방콕 호텔간 짐... 정보 없음 정보 없음         처리 안됨 https://experiences.myrealt... 2025-07-13 01:38:09 완전수집

📈 수집 통계:
   🌏 대륙별 분포: {'아시아': 1}
   🏙️ 국가별 분포: {'태국